In [1]:
import networkx as nx
from networkx import DiGraph
from networkx import dijkstra_path

In [2]:
class Region(object):
    def __init__(self, coord, app, name=None):
        ## ap is a LIST!!!
        self.coord = coord
        self.name = name
        self.app = app
        self.apr = [self.name]
        self.ap = self.app + self.apr

    def __repr__(self):
        return self.name

In [3]:
r4 = Region((0,2),['basket'],'r4')
r5 = Region((1,2),['rball'],'r5')
r6 = Region((2,2),[],'r6')
c1 = Region((0,1),[],'c1')
c2 = Region((1,1),[],'c2')
c3 = Region((2,1),[],'c3')
r1 = Region((0,0),[],'r1')
r2 = Region((1,0),['basket'],'r2')
r3 = Region((2,0),['gball'],'r3')

region_list = [r1, r2, r3, r4, r5, r6, c1, c2, c3]

In [4]:
class wFTS(object):
    def __init__(self, states=set(), transition={}, initial=set()):
        # trans_relat is a matrix stores control words
        ## i.e. trans_relat[2][4] 
        # weight is a matrix stores transition cost if reachable, otherwise +infty
        ## i.e. trans_relat[4][9] represents the weight when travel from region 5 to region 10
        self.states = states
        # states stores Region objects
        self.transition = transition
        self.initial = initial
        
    def add_states(self,new):
        self.states.add(new)
        if new not in self.transition.keys():
            self.transition[new] = {}
            self.transition[new][new] = 0
        else:
            raise AttributeError('This node is already in states')
        
    def add_initial(self,new):
        self.initial.add(new)
        
    def add_transition(self,state1,state2,w=0,symmetric=True):
        if state1 not in self.transition.keys():
            self.transition[state1] = {}
        if state2 not in self.transition.keys():
            self.transition[state2] = {}
        self.transition[state1][state2] = w
        if symmetric == True:
            self.transition[state2][state1] = w
        
    def change_weight(self,state1,state2,w,symmetric=True):
        self.transition[state1][state2] = w
        if symmetric == True:
            self.transition[state2][state1] = w
        
    def L(self,region):
        return region.ap
    
    def get_weight(self, region1, region2):
        if region2 in self.transition[region1].keys():
            return self.transition[region1][region2]
        else:
            raise ValueError('Cannot transit between %s and %s' %(str(region1.name),str(region2.name)) )
    
    def post(self,region):
        if region not in self.states:
            raise AttributeError('State not valid')
        else:
            return [i for i in self.transition[region].keys()]
        

In [5]:
wfts = wFTS()

for i in region_list:
    wfts.add_states(i)
    
wfts.add_transition(r1,c1,1)
wfts.add_transition(c1,c2,1)
wfts.add_transition(c1,r4,1)
wfts.add_transition(c2,c3,1)
wfts.add_transition(c2,r2,1)
wfts.add_transition(c2,r5,1)
wfts.add_transition(c3,r6,1)
wfts.add_transition(c3,r3,1)

wfts.add_initial(r1)




In [6]:
import graphviz as gv
wfts_graph = DiGraph()
wfts_graph.add_nodes_from([i for i in wfts.states])
for i in wfts.states:
    for j in wfts.transition[i].keys():
        if wfts.transition[i][j] is not None:
            wfts_graph.add_edge(i,j,weight=wfts.transition[i][j])
nx.write_dot(wfts_graph,'./wfts.dot')
with open('./wfts.dot','r') as myfile:
    plot_wfts_graph = myfile.read().replace('\n','').replace('weight','label')
plot_wfts_graph = gv.Source(wfts_graph)
plot_wfts_graph.render('./wfts.dot',view=True)


'./wfts.dot.pdf'

In [7]:
class Buchi_Automaton(object):
    def __init__(self, buchi):
        self.states = [i for i in buchi.nodes()]
        # states are str
        self.alphabet = [i for i in buchi.graph['symbols']]
        self.transition = buchi.edge
        self.initial = [i for i in buchi.graph['initial']]
        self.accept = [i for i in buchi.graph['accept']]
        
    def get_ap(self, state1, state2):
        if state1 not in self.states or state2 not in self.states:
            raise AttributeError('State not valid')
        elif state2 not in self.transition[state1].keys():
            raise AttributeError('State2 cannot be reached from State1')
        else:
            result = self.transition[state1][state2]['guard_formula']
            if len(result) == 1:
                return result
            else:    
                return result[1:-1]
        
    def post(self, state):
        if state not in self.states:
            raise AttributeError('State not valid')
        else:
            return [i for i in self.transition[state].keys()]
        
    def get_transition_through_ap(self, state, ap):
        if '&&' in ap:
            reachable_state = []
            seperated_ap,neg_in_ap = self.seperate_ap_sentence(ap)
            for i in self.transition[state].keys():
                j = self.get_ap(state,i)
                seperated_j,neg_in_j = self.seperate_ap_sentence(j)
                if '&&' in j:
                    if len(seperated_ap) >= len(seperated_j):
                        if set(seperated_ap).issuperset(set(seperated_j)) and self.check_negations(seperated_j,neg_in_j,seperated_ap,neg_in_ap):
                            reachable_state += [i]
                else:
                    if '!' in j:
                        if j in ap or self.check_negations(seperated_j,neg_in_j,seperated_ap,neg_in_ap):
                            reachable_state += [i]
                    else:
                        if j in ap or j=='1':
                            reachable_state += [i]
            return reachable_state
                        
        else:
            reachable_state = []
            seperated_ap,neg_in_ap = self.seperate_ap_sentence(ap)
            for i in self.transition[state].keys():
                j = self.get_ap(state,i)
                seperated_j,neg_in_j = self.seperate_ap_sentence(j)
                if '&&' in j:
                    if len(seperated_j) <= 1:
                        if set(seperated_ap).issuperset(set(seperated_j)) and self.check_negations(seperated_j,neg_in_j,seperated_ap,neg_in_ap):
                            reachable_state += [i] 
                else:
                    if '!' not in j:
                        if j in ap or j=='1':
                            reachable_state += [i]
                    else:
                        if '!' in ap:
                            if j == ap:
                                reachable_state += [i]
                        else:
                            if self.check_negations(seperated_j,neg_in_j,seperated_ap,neg_in_ap):
                                reachable_state += [i]
            return reachable_state


    def plot(self,filename='current_buchi'):
        plot_buchi(buchi,filename)
        
    def find_ampersand(self,input_str):
        index = []
        original_length = len(input_str)
        original_str = input_str
        while input_str.find('&&')>=0:
            index += [input_str.index('&&')-len(input_str)+original_length]
            input_str = original_str[index[-1]+2:]
        return index
    
    def seperate_ap_sentence(self,input_str):
        if len(input_str)>1:
            index = self.find_ampersand(input_str)
            if len(index)>=1:
                return_str = [input_str[0:index[0]]]
            else:
                return_str = input_str
                if '!' in return_str:
                    return [],[return_str.replace('!','')]
                else:
                    return [return_str],[]
            for i in range(1,len(index)):
                return_str += [input_str[index[i-1]+2:index[i]]]
            return_str = return_str + [input_str[index[-1]+2:]]
            return_str = [i.replace(' ','') for i in return_str]
        elif len(input_str)==1:
            return_str = input_str
        elif len(input_str) == 0:    
            raise AttributeError('input_str has no length')
            
        without_negs = []
        negations = []
        for i in range(len(return_str)):
            if '!' in return_str[i]:
                negations += [return_str[i].replace('!','')]
            else:
                without_negs += [return_str[i]]
        return without_negs,negations
    
    def check_negations(self,set1,neg1,set2,neg2):
        for i in set1:
            if i in neg2:
                return False
        for i in set2:
            if i in neg1:
                return False
        return True
        

In [8]:
import sys
sys.path.append('/Users/gaoqitong/Master_Project/Thesis_Code/')
sys.path.append('/Users/gaoqitong/Master_Project/P_MAS_TG-master/')
from P_MAS_TG.buchi import buchi_from_ltl
formula = '<>(rball && <>basket) && <>[]r1'
# formula = '<>(rball && <>basket) && <>(gball && <>basket) && <>[]r1 && [](rball->X(!gball U basket)) && [](gball->X(!rball U basket))'
# formula = '<>(rball && <>(basket && r2)) && <>(gball && <>(basket && r4)) && [](rball->X(!gball U basket)) && [](gball->X(!rball U basket)) && <>[]r1'

buchi = buchi_from_ltl(formula,None)
my_buchi = Buchi_Automaton(buchi)

from VisualizeBuchi import plot_buchi
my_buchi.plot()


Graph Successfully Plotted


In [9]:
class FullProd(object):
    def __init__(self, wFTS, Buchi):
        self.states = set()
        self.transition = {}
        # transition is a dict where every state stores as a key and value is a dic contains every reachable
        # state from this key and the weight to transit
        self.initial = set()
        self.accept = set()
        self.wFTS = wFTS
        self.Buchi = Buchi
        
    def construct_fullproduct(self):
        alpha = 1
        for pi_i in self.wFTS.states:
            for q_m in self.Buchi.states:
                q_s = (pi_i,q_m)
                self.states.add(q_s)
                if q_s not in self.transition.keys():
                    self.transition[q_s] = {}
                if pi_i in self.wFTS.initial and q_m in self.Buchi.initial:
                    self.initial.add(q_s)
                if q_m in self.Buchi.accept:
                    self.accept.add(q_s)
                    
                for pi_j in self.wFTS.post(pi_i):
                    for q_n in self.Buchi.post(q_m):
                        q_g = (pi_j,q_n)
                        self.states.add(q_g)
                        if q_g not in self.transition.keys():
                            self.transition[q_g] = {}
                        d = self.check_tran_b(q_m,self.wFTS.L(pi_i),q_n,self.Buchi)
                        if d >= 0:
                            self.transition[q_s][q_g] = self.wFTS.get_weight(pi_i,pi_j) + alpha*d
                            
    def get_weight(self,state1,state2):
        return self.transition[state1][state2]

                        
    def check_tran_b(self, b_state1, l, b_state2, Buchi):
        d = -1
        if len(l) == 1:
            if b_state2 in self.Buchi.get_transition_through_ap(b_state1, l[0]):
                d = 0
                return d
        if len(l) > 1:
            conjunction = [l[i]+'&&' if i!=len(l)-1 else l[i] for i in range(len(l))]
            conjunction = ''.join(conjunction)
            if b_state2 in self.Buchi.get_transition_through_ap(b_state1, conjunction):
                d = 0
                return d
        return d
    
    def get_state(self,index):
        return list(self.states)[index]
    
    def return_graph(self):
        index = 0
        digraph = DiGraph()
        digraph.add_nodes_from([i for i in self.states])
        for i in self.states:
            for j in self.transition[i].keys():
                if self.transition[i][j] is not None:
                    digraph.add_edge(i,j,weight=self.transition[i][j])
        return digraph
                        

In [10]:
full_prod = FullProd(wfts,my_buchi)
full_prod.construct_fullproduct()
count = 0
for i in full_prod.states:
    for j in full_prod.transition[i].keys():
        if full_prod.transition[i][j] is not None:
            count += 1            
print count

153


In [11]:
from networkx import DiGraph
import networkx as nx
import graphviz as gv
graph = full_prod.return_graph()
nx.write_dot(graph,'./FullProd.dot')
with open('./FullProd.dot','r') as myfile:
    graph = myfile.read().replace('\n','').replace('weight','label')
graph = gv.Source(graph)
graph.render('./FullProd.dot',view=True)

'./FullProd.dot.pdf'

In [12]:
def search_opt_run(FullProduct):
    candidates_pre = {}
    candidates_suf = {}
    candidates = {}
    G = FullProduct.return_graph()
    for initial_state in FullProduct.initial:
        for accept_state in FullProduct.accept:
            try:
                opt_path = nx.dijkstra_path(G,initial_state,accept_state,'weight')
                path_cost = nx.dijkstra_path_length(G,initial_state,accept_state,'weight')
                if initial_state not in candidates_pre.keys():
                    candidates_pre[initial_state] = {}
                candidates_pre[initial_state][accept_state] = (opt_path,path_cost)
            except:
                pass
#     print candidates_pre
    for accept_state in FullProduct.accept:
        successors = FullProduct.transition[accept_state].keys()
        best_path = []
        best_cost = float('inf')
        for successor in successors:
            if successor is not accept_state:
                try:
                    current_path = nx.dijkstra_path(G,accept_state,successor,'weight') + nx.dijkstra_path(G,successor,accept_state,'weight')
                    current_cost = nx.dijkstra_path_length(G,accept_state,successor,'weight') + nx.dijkstra_path_length(G,successor,accept_state,'weight')
                    if current_cost < best_cost:
                        best_path = current_path
                        best_cost = current_cost
                except:
                    pass
            else:
                current_path = [(accept_state,accept_state)]
                current_cost = FullProduct.transition[accept_state][accept_state]
                if current_cost < best_cost:
                    best_path = current_path
                    best_cost = current_cost
        if best_cost < float('inf'):
            candidates_suf[accept_state] = (best_path,best_cost)
#     print candidates_suf
    for initial_state in candidates_pre.keys():
        for accept_state in candidates_pre[initial_state].keys():
            if accept_state in candidates_suf.keys():
                candidates[(initial_state,accept_state)] = (candidates_pre[initial_state][accept_state][0]+candidates_suf[accept_state][0],
                                                           candidates_pre[initial_state][accept_state][1]+candidates_suf[accept_state][1])
    opt_run = min(candidates.items(),key=lambda x : x[1][1])
    return opt_run[1]

In [13]:
opt=search_opt_run(full_prod)
# print graph.edge[(region4,'accept_S6')]
# print nx.find_cycle(graph,(region1,'accept_S6'))

In [14]:
print [opt[0][i][0] for i in range(len(opt[0]))]
print len([opt[0][i][0] for i in range(len(opt[0]))])

[r1, c1, c2, r5, c2, c1, r4, c1, r1, r1, r1, r1]
12
